# Block descent for optimizing group knockoffs

There are 2 ways for optimizing the group knockoff problem. This notebook tests the block update strategy where we update an entire block at once. Two concerns arise 

+ Which solver (MOSEK, Hypatia.jl, SCS, SDPT3, Convex.jl) should we use to solve each individual block?
+ How to maintain inequality constraints after updating a block? Can we use Woodbury?

In [217]:
using Revise
using Knockoffs
using BlockDiagonals
using LinearAlgebra
using Hypatia
using JuMP
using ToeplitzMatrices
using StatsBase
# ENV["COLUMNS"] = 240

#
# test problem
#
p = 200
group_sizes = [5 for i in 1:div(p, 5)] # each group has 5 variables
groups = vcat([i*ones(g) for (i, g) in enumerate(group_sizes)]...) |> Vector{Int}
# Σ = simulate_block_covariance(groups, 0.75, 0.25)
# Σ = Matrix(SymmetricToeplitz(0.4.^(0:(p-1)))) # true covariance matrix
# Σ = Matrix(SymmetricToeplitz((-0.4).^(0:(p-1)))) # true covariance matrix
# Σ = simulate_block_covariance(groups, 0.5, 0.6) # true covariance matrix
Σ = simulate_AR1(p, a=3, b=1) # true covariance matrix
# x = randn(p, p)
# Σ = x'*x
# StatsBase.cov2cor!(Σ, std(Σ, dims=1))
# Σ

200×200 Matrix{Float64}:
 1.0          0.665309     0.482002     …  3.06574e-29  2.51951e-29
 0.665309     1.0          0.724478        4.60799e-29  3.78697e-29
 0.482002     0.724478     1.0             6.36043e-29  5.22717e-29
 0.459989     0.691391     0.95433         6.66481e-29  5.47732e-29
 0.39511      0.593875     0.819727        7.7592e-29   6.37672e-29
 0.296705     0.445965     0.615567     …  1.03326e-28  8.49164e-29
 0.279914     0.420729     0.580733        1.09524e-28  9.00099e-29
 0.185685     0.279096     0.385237        1.65104e-28  1.35687e-28
 0.099644     0.149771     0.20673         3.07669e-28  2.52851e-28
 0.0804874    0.120977     0.166986        3.80897e-28  3.13031e-28
 0.0260932    0.0392197    0.0541351    …  1.17492e-27  9.65579e-28
 0.0171078    0.0257141    0.0354933       1.79201e-27  1.47272e-27
 0.0164554    0.0247335    0.0341398       1.86305e-27  1.53111e-27
 ⋮                                      ⋱               
 5.98244e-28  8.99198e-28  1.24117

Try block descent

In [218]:
@time Ssdp, _ = solve_s_group(
    Σ, groups, :sdp_block,
    m = 1, 
    verbose=true,
    niter=5
);

Init obj = 628.8147402770132
Init block objs = [17.14365532259672, 14.54900852238708, 17.245009276869844, 13.758584626815354, 18.487119323297673, 13.985812175272535, 12.537336881227928, 9.978489930050154, 14.555484307457197, 17.28545044720164, 18.015229328219, 18.6540776912254, 15.745721698144473, 20.716620262906176, 16.004909633335508, 16.035436073255628, 16.120674404774736, 14.550779269001165, 20.014126331093056, 14.467364010369023, 18.396306231587058, 14.980949354007999, 14.92153817244236, 14.897367292906418, 12.755291349150529, 15.551675597114953, 17.292902095569865, 17.027253770399447, 22.269168609281728, 13.53640726132273, 15.776788544721612, 13.405937591303235, 18.284455667627768, 14.21434063068739, 12.96444321573064, 13.191732583243992, 15.601085963834755, 13.719451276367565, 16.267784030894287, 13.908971523318636]
Iter 1 block 1 old objective = 17.14365532259672 and new obj = 0.30857895322346596
Iter 1 block 2 old objective = 14.54900852238708 and new obj = 0.6408172517565853


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 1 block 3 old objective = 17.245009276869844 and new obj = 0.9273074555671013


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 1 block 4 old objective = 13.758584626815354 and new obj = 2.038010643199576
Iter 1 block 5 old objective = 18.487119323297673 and new obj = 11.507481751856343
Iter 1 block 6 old objective = 13.985812175272535 and new obj = 3.597559927752086
Iter 1 block 7 old objective = 12.537336881227928 and new obj = 9.35709113387171
Iter 1 block 8 old objective = 9.978489930050154 and new obj = 10.188919778116885
Iter 1 block 9 old objective = 14.555484307457197 and new obj = 1.5331392143004465


...

┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 2 block 19 old objective = 2.3649045001358053 and new obj = 2.3649045001358053
Iter 2 block 20 old objective = 4.26163210960868 and new obj = 4.26163210960868


┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


Iter 2 block 21 old objective = 10.446039944105223 and new obj = 10.446039944105223
Iter 2 block 22 old objective = 6.509187381717455 and new obj = 6.509187381717455
Iter 2 block 23 old objective = 3.6407707655476544 and new obj = 3.6407707655476544
Iter 2 block 24 old objective = 8.205597222821185 and new obj = 8.205597222821185
Iter 2 block 25 old objective = 7.7629113718076646 and new obj = 7.7629113718076646
Iter 2 block 26 old objective = 9.140896530116445 and new obj = 9.140896530116445
Iter 2 block 27 old objective = 13.923782556033277 and new obj = 13.923782556033277
Iter 2 block 28 old objective = 7.449514067787803 and new obj = 7.449514067787803
Iter 2 block 29 old objective = 9.711870559799639 and new obj = 9.711870559799639
Iter 2 block 30 old objective = 7.244967749088241 and new obj = 7.244967749088241
Iter 2 block 31 old objective = 7.233280201877542 and new obj = 7.233280201877542
Iter 2 block 32 old objective = 13.516453074723145 and new obj = 13.516453074723145
Iter 2

┌ Warning: cannot step in centering direction
└ @ Hypatia.Solvers /Users/biona001/.julia/packages/Hypatia/gNjn6/src/Solvers/steppers/combined.jl:106


In [219]:
@time Sequi, _ = solve_s_group(
    Σ, groups, :equi,
    m = 1, 
);

  0.012215 seconds (1.29 k allocations: 2.106 MiB)


In [220]:
@time Sme, _ = solve_s_group(
    Σ, groups, :maxent,
    m = 1, 
    verbose=true
);

initial obj = -913.0787833855966
Iter 1: obj = -639.7152194681657, δ = 0.7080051874381286, t1 = 0.01, t2 = 0.01, t3 = 0.0
Iter 2: obj = -626.533436464895, δ = 0.21521364307041269, t1 = 0.02, t2 = 0.02, t3 = 0.0
Iter 3: obj = -623.1639029190135, δ = 0.027188857061293814, t1 = 0.03, t2 = 0.04, t3 = 0.0
Iter 4: obj = -621.764132451856, δ = 0.00672527262946665, t1 = 0.04, t2 = 0.05, t3 = 0.0
Iter 5: obj = -621.0512717150402, δ = 0.01736959430176399, t1 = 0.05, t2 = 0.06, t3 = 0.0
Iter 6: obj = -620.630378975075, δ = 0.005999679589528073, t1 = 0.06, t2 = 0.07, t3 = 0.0
Iter 7: obj = -620.3598296436713, δ = 0.002898884647958585, t1 = 0.06, t2 = 0.08, t3 = 0.0
Iter 8: obj = -620.1728919728496, δ = 0.0026129221795445842, t1 = 0.07, t2 = 0.1, t3 = 0.0
Iter 9: obj = -620.0426403982601, δ = 0.0022623510828712295, t1 = 0.08, t2 = 0.11, t3 = 0.0
Iter 10: obj = -619.9410126316154, δ = 0.001915924309315557, t1 = 0.09, t2 = 0.12, t3 = 0.0
Iter 11: obj = -619.8646422029101, δ = 0.0015864395071964559, t

In [221]:
sum(Sequi), sum(Ssdp), sum(Sme)

(42.05914631656778, 507.43573050191003, 68.89354910569924)

In [222]:
[vec(Sequi) vec(Ssdp) vec(Sme)]

40000×3 Matrix{Float64}:
 0.0626931  1.0       0.555897
 0.0417102  0.665309  0.184368
 0.0302182  0.482002  0.0209709
 0.0288381  0.459989  0.00883674
 0.0247707  0.39511   0.00441798
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 ⋮                    
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0        0.0       0.0
 0.0130953  0.39511   0.000925487
 0.0167721  0.593875  0.00262455
 0.0186497  0.819727  0.00893611
 0.0515229  0.858956  0.169722
 0.0626931  1.0       0.32336

What if we run group knockoffs when all groups are singletons?

In [127]:
groups = collect(1:p)

# group SDP with single groups
@time Ssdp, _ = solve_s_group(
    Σ, groups, :sdp_block,
    m = 1, 
    verbose=true,
    niter=5
)

# non-grouped SDP
@time ssdp = solve_s(
    Symmetric(Σ), :sdp,
    m = 1, 
    verbose=true,
);

g = 1
A11 = [2.0;;]
size(A11) = (1, 1)


LoadError: UndefVarError: fff not defined

Here Ssdp should be diagonal matrix.

In [87]:
Ssdp

200×200 Matrix{Float64}:
 1.0  0.0  0.0  0.0       0.0      0.0       …  0.0       0.0       0.0
 0.0  1.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  1.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.995473  0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.63106  0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.554254  …  0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0       …  0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 0.0  0.0  0.0  0.0       0.0      0.0          0.0       0.0       0.0
 ⋮                                 ⋮   

In [86]:
[ssdp diag(Ssdp)]

200×2 Matrix{Float64}:
 1.0         1.0
 0.468434    1.0
 0.86833     1.0
 0.914276    0.995473
 3.36244e-8  0.63106
 0.5571      0.554254
 4.10019e-8  0.298628
 0.236487    0.265939
 0.0911048   0.198897
 0.0657967   0.186742
 0.198103    0.351992
 0.29984     0.611173
 0.694426    0.84924
 ⋮           
 0.0318133   0.047177
 0.011571    0.0522678
 0.599472    0.822529
 0.412487    0.965399
 0.819467    0.90014
 1.32504e-8  0.398157
 0.354499    0.437457
 0.403364    0.483912
 1.07897e-8  0.136476
 0.115996    0.112647
 1.02813e-8  0.181416
 0.376427    0.371388

In [89]:
eigmin(2Σ - Diagonal(ssdp))

-4.075475130804836e-9

In [91]:
eigmin(2Σ - Ssdp)

-0.3749872292309029

## How to permute things

In [224]:
S1 = reshape(1:16, 4, 4) |> Matrix
S2 = reshape(1:9, 3, 3) |> Matrix
S3 = reshape(1:4, 2, 2) |> Matrix
S = BlockDiagonal([S1, S2, S3])

9×9 BlockDiagonal{Int64, Matrix{Int64}}:
 1  5   9  13  0  0  0  0  0
 2  6  10  14  0  0  0  0  0
 3  7  11  15  0  0  0  0  0
 4  8  12  16  0  0  0  0  0
 0  0   0   0  1  4  7  0  0
 0  0   0   0  2  5  8  0  0
 0  0   0   0  3  6  9  0  0
 0  0   0   0  0  0  0  1  3
 0  0   0   0  0  0  0  2  4

In [233]:
perm = collect(1:9)
g = 2
offset = 7
cur_idx = offset + 1:offset + g
for i in 1:offset
    perm[g+i] = i
end
perm[1:g] .= cur_idx
perm

9-element Vector{Int64}:
 8
 9
 1
 2
 3
 4
 5
 6
 7

In [235]:
Spermuted = S[perm, perm]

9×9 Matrix{Int64}:
 1  3  0  0   0   0  0  0  0
 2  4  0  0   0   0  0  0  0
 0  0  1  5   9  13  0  0  0
 0  0  2  6  10  14  0  0  0
 0  0  3  7  11  15  0  0  0
 0  0  4  8  12  16  0  0  0
 0  0  0  0   0   0  1  4  7
 0  0  0  0   0   0  2  5  8
 0  0  0  0   0   0  3  6  9

In [236]:
iperm = invperm(perm)
Spermuted[iperm, iperm]

9×9 Matrix{Int64}:
 1  5   9  13  0  0  0  0  0
 2  6  10  14  0  0  0  0  0
 3  7  11  15  0  0  0  0  0
 4  8  12  16  0  0  0  0  0
 0  0   0   0  1  4  7  0  0
 0  0   0   0  2  5  8  0  0
 0  0   0   0  3  6  9  0  0
 0  0   0   0  0  0  0  1  3
 0  0   0   0  0  0  0  2  4

## Solving Convex problems

Try solving first block

In [157]:
#
# test problem
#
p = 15
group_sizes = [5 for i in 1:div(p, 5)] # each group has 5 variables
groups = vcat([i*ones(g) for (i, g) in enumerate(group_sizes)]...) |> Vector{Int}
# Σ = simulate_block_covariance(groups, 0.75, 0.25)
# Σ = Matrix(SymmetricToeplitz(0.4.^(0:(p-1)))) # true covariance matrix
# Σ = Matrix(SymmetricToeplitz((-0.4).^(0:(p-1)))) # true covariance matrix
Σ = simulate_AR1(p, a=3, b=1) # true covariance matrix

"""
    solve_full_SDP
"""
function solve_full_SDP(
    Σ11::AbstractMatrix, # correlation matrix
    ub::AbstractMatrix; # this is [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, -1 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    # slack variables to handle absolute value in obj 
    @variable(model, U[1:p, 1:p], Symmetric)
    for i in 1:p, j in i:p
        @constraint(model, Σ11[i, j] - S[i, j] ≤ U[i, j])
        @constraint(model, -U[i, j] ≤ Σ11[i, j] - S[i, j])
    end
#     @constraint(model, U in PSDCone())      #### is this constraint needed????
    @objective(model, Min, sum(U))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S), objective_value(model)
end

function solve_full_SDP_old(
    Σ11::AbstractMatrix, # correlation matrix
    ub::AbstractMatrix; # this is [A12 A13]*inv(A22-S2 A32; A23 A33-S3)*[A21; A31]
    optm=Hypatia.Optimizer(verbose=false) # Any solver compatible with JuMP
    )
    # Build model via JuMP
    p = size(Σ11, 1)
    model = Model(() -> optm)
    @variable(model, 0 ≤ S[1:p, 1:p] ≤ 1, Symmetric)
    @objective(model, Min, sum(Σ11 - S))
    # SDP constraints
    @constraint(model, S in PSDCone())
    @constraint(model, ub - S in PSDCone())
    # solve and return
    JuMP.optimize!(model)
    return JuMP.value.(S)
end

solve_full_SDP_old (generic function with 1 method)

In [158]:
m = 1

# initialize with equicorrelated solution
# Sequi, γ = solve_s_group(Σ, groups, :equi)
Sequi = zeros(p, p)

# form constraints for block 1
Σ11 = Σ[1:5, 1:5]
A = (m+1)/m * Σ
D = A - Sequi
A11 = @view(A[1:5, 1:5])
D12 = @view(D[1:5, 6:end])
D22 = @view(D[6:end, 6:end])
ub = A11 - D12 * inv(D22) * D12'

# solve 
@time S1_new, obj = solve_full_SDP(Σ11, ub)
@time S1_old = solve_full_SDP_old(Σ11, ub);

  0.223723 seconds (456.79 k allocations: 24.844 MiB, 83.68% compilation time)
  0.109151 seconds (279.45 k allocations: 15.444 MiB, 77.42% compilation time)


In [163]:
obj, sum(abs.(Σ11 - S1_new))

(2.696897785289176e-9, 9.016139224105046e-10)

In [160]:
eigmin(S1_new)

0.0790755108995826

In [161]:
eigmin(ub)

0.1581510007508361

Lets eyeball the result first

In [94]:
S1_new

5×5 Matrix{Float64}:
 1.0     0.4    0.16  0.064  0.0256
 0.4     1.0    0.4   0.16   0.064
 0.16    0.4    1.0   0.4    0.16
 0.064   0.16   0.4   1.0    0.4
 0.0256  0.064  0.16  0.4    1.0

In [95]:
S1_old

5×5 Matrix{Float64}:
 0.789886  0.747998  0.769968  0.644723  0.319591
 0.747998  0.962284  0.938245  0.82901   0.502877
 0.769968  0.938245  0.982537  0.854499  0.525783
 0.644723  0.82901   0.854499  0.879502  0.417846
 0.319591  0.502877  0.525783  0.417846  0.477052

In [96]:
sum(abs.(S1_new)), sum(abs.(S1_old))

(9.467199998341476, 17.192337396303046)

Check objective

In [162]:
sum(abs.(Σ11 - S1_new)), sum(abs.(Σ11 - S1_old))

(9.016139224105046e-10, 8.45111995244404)

Check constraints

In [98]:
isposdef(S1_old), isposdef(S1_new)

(true, true)

In [99]:
eigvals(ub - S1_old)

5-element Vector{Float64}:
 7.309755484523514e-9
 0.7520353697688693
 0.9080664264148712
 1.4359163255019285
 2.0506742273768

In [100]:
eigvals(ub - S1_new)

5-element Vector{Float64}:
 0.32129497088496595
 0.4766120211625212
 0.643214501447116
 1.0285108585981437
 1.7683208812336677

### Singleton groups?

In [105]:
groups[2:5] .= 2
groups

15-element Vector{Int64}:
 1
 2
 2
 2
 2
 2
 2
 2
 2
 2
 3
 3
 3
 3
 3

In [132]:
m = 1

# initialize with equicorrelated solution
Sequi, γ = solve_s_group(Σ, groups, :equi)

# form constraints for block 1
g = 1
Σ11 = @view(Σ[1, 1])
A = (m+1)/m * Σ
D = A - Sequi
Σ11 = @view(Σ[1:g, 1:g])
A11 = @view(A[1:g, 1:g])
D12 = @view(D[1:g, g + 1:end])
D21 = @view(D[g + 1:end, 1:g])
D22 = @view(D[g + 1:end, g + 1:end])
ub = Symmetric(A11 - D21' * inv(D22) * D21)

# solve 
@time S1_new, U = solve_full_SDP(Σ11, ub)
# @time S1_old = solve_full_SDP_old(Σ11, ub);

  0.066523 seconds (118.17 k allocations: 6.466 MiB, 89.63% compilation time)


In [134]:
S1_new, ub

([0.8644309427462861;;], [0.8644309426414065;;])

## Woodbury updates

Suppose we have 

\begin{align*}
    S = 
    \begin{bmatrix}
        S^{(1)} & & \\
        & S^{(2)} & \\
        & & S^{(3)}
    \end{bmatrix}
\end{align*}
And we want to udpate $S^{(1)}$

$$(A + UC)^{-1} = A^{-1} - A^{-1}U(I + VA^{-1}U)^{-1}VA^{-1}$$

In [1]:
using WoodburyMatrices

In [ ]:
A = 